In [2]:
from PyPDF2 import PdfReader
import re
import os
import json
import openai
from pprint import pprint
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from dotenv import load_dotenv
from langchain.schema import Document
import os
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain.schema import Document
from fastembed import LateInteractionTextEmbedding
from qdrant_client.models import PointStruct, SparseVector
from qdrant_client.models import VectorParams, Distance
from qdrant_client import QdrantClient, models
from langchain_core.documents import Document
from pprint import pprint


In [ ]:
# Initial text extraction. This function extracts text from a PDF file, starting after a specific phrase and stopping at another phrase. It also skips unreadable or empty pages and stops processing at "Annex B" but only if it's after the defined minimum page.
def extract_text(file_path, start_after, stop_at, stop_at_min_page=10):
    reader = PdfReader(file_path)
    text = ""
    has_started = False

    for i, page in enumerate(reader.pages):
        page_text = page.extract_text()

        # Skip unreadable or empty pages
        if not page_text or page_text.strip() == "":
            continue

        # Start extracting after detecting the start phrase
        if not has_started and start_after in page_text:
            has_started = True
            continue

        # Stop processing at "Annex B" but only if it's after the defined minimum page
        if has_started and stop_at in page_text and i >= stop_at_min_page:
            break

        # Append text if extraction has started
        if has_started:
            text += page_text

    return text


def clean_and_split_text(text):
    # Remove license and metadata
    text = re.sub(r'Licensed to .*?prohibited\.', '', text, flags=re.DOTALL)
    text = re.sub(r'© ISO \d{4}.*?\n', '', text)
    text = re.sub(r'© ISO/IEC \d{4}.*?\n', '', text)


    # Normalize spaces but preserve newlines for sections
    text = re.sub(r'\s{2,}', ' ', text)  
    text = re.sub(r'(\d+\.\d+)\s([A-Z])', r'\1\n\2', text)  
    text = re.sub(r'(\d+\sContext)', r'\n\1', text)  
    return text

def chunk_text_by_sections(text):
    chunks = {}
    current_section = None
    # section_pattern = re.compile(r'^(\d+(\.\d+)*)(\s+.*)?$')  
    section_pattern = re.compile(r'^((\d+|[A-Z])(\.\d+)*)(\s.*)?$')

    lines = text.splitlines()

    for line in lines:
        line = line.strip()
        match = section_pattern.match(line)
        if match:
            # Start a new section
            section_header = match.group(1)
            current_section = section_header
            chunks[current_section] = match.group(0).strip() + "\n"
        elif current_section:
            # Add content to the current section
            chunks[current_section] += line + " "

    # Clean up chunk content
    for key in chunks:
        chunks[key] = chunks[key].strip()

    return chunks

In [ ]:

# Main execution
def chunk_ISO_sections(folder_path, start_phrase, stop_phrase):

    


    pdf_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".pdf")]


    all_chunks = {}

    for file in pdf_files:
        raw_text = extract_text(file, start_phrase, stop_phrase)
        cleaned_text = clean_and_split_text(raw_text)
        chunks = chunk_text_by_sections(cleaned_text)
        all_chunks[file] = chunks

    
    return all_chunks
    
    

folder_path = "ISO standards"
start_phrase = "ISO (the International Organization for Standardization) is a worldwide federation"
stop_phrase = "Annex B"
all_chunks = chunk_ISO_sections(folder_path, start_phrase, stop_phrase)
print(all_chunks)

{'ISO standards\\ISO_14001_2015(en).pdf': {'0.1': '0.1\nBackground Achieving a balance between the environment, society and the economy is considered essential to meet the needs of the present without compromising the ability of future generations to meet their needs. Sustainable development as a goal is achieved by balancing the three pillars of sustainability. Societal expectations for sustainable development, transparency and accountability have evolved with increasingly stringent legislation, growing pressures on the environment from pollution, inefficient use of resources, improper waste management, climate change, degradation of ecosystems and loss of biodiversity. This has led organizations to adopt a systematic approach to environmental management by implementing environmental management systems with the aim of contributing to the environmental pillar of sustainability.', '0.2': '0.2\nAim of an environmental management system The purpose of this International Standard is to pro

In [ ]:
# ISO requirements tagging
def tag_section_requirement(text):
    """Tags sections based on keywords indicating requirements, recommendations, or permissions."""
    # First shall so if both shall and should are in the text, it will be tagged as a requirement
    if "shall" in text.lower():
        return "Requirement"
    elif "should" in text.lower():
        return "Recommendation"
    elif "may" in text.lower():
        return "Permission"
    else:
        return "General"


# ISO section chunking
def process_document(data):
    """Processes a document and tags sections with requirement tags."""
    documents = []
    default_parent_section = {

        "section_id": "Not correctly extracted",
        "text": "Not correctly extracted",
        "title": "Not correctly extracted"
        
    }
    
    for doc, sections in data.items():
        parent_section = None
        for section_id, text in sections.items():
            requirement_tag = tag_section_requirement(text)
            
            # Proceed with the rest of your section processing
            if section_id.startswith('A') or section_id.startswith('B'):  
                if '.' not in section_id:
                    parent_section = {
                        "section_id": section_id,
                        "title": text.split("\n", 1)[0] if "\n" in text else text,
                        "requirement_tag": requirement_tag,
                        "text": text,
                        "source": doc
                        
                    }
                    documents.append(parent_section)
                else:
                    matching_parent = next(
                        (doc for doc in documents if doc["section_id"] == section_id.rsplit('.', 1)[0]),
                        None
                    )
                    if matching_parent:
                        documents.append({
                            "section_id": section_id,
                            "parent_id": matching_parent["section_id"],
                            "parent_title": matching_parent.get("title", ""),
                            "requirement_tag": requirement_tag,
                            "text": text,
                            "source": doc
                            
                        })
            else:  # Normal numbered sections
                if '.' not in section_id:
                    parent_section = {
                        "section_id": section_id,
                        "requirement_tag": requirement_tag,
                        "title": text.split("\n", 1)[0] if "\n" in text else text,
                        "text": text,
                        "source": doc
                        
                    }
                    documents.append(parent_section)
                else:
                    matching_parent = next(
                        (doc for doc in documents if doc["section_id"] == section_id.rsplit('.', 1)[0]),
                        None
                    )
                    if matching_parent:
                        documents.append({
                            "section_id": section_id,
                            "parent_id": matching_parent["section_id"],
                            "parent_title": matching_parent.get("title", ""),
                            "requirement_tag": requirement_tag,
                            "text": text,
                            "source": doc
                            
                        })
                    else:
                        parent_id = parent_section["section_id"] if parent_section else default_parent_section["section_id"]
                        documents.append({
                            "section_id": section_id,
                            "parent_id": parent_id,
                            "parent_title": parent_section["title"] if parent_section else default_parent_section["title"],
                            "requirement_tag": requirement_tag,
                            "text": text,
                            "source": doc,
                            
                        })
    return documents




In [8]:
all_iso_chunked = process_document(all_chunks)


In [12]:
pprint(all_iso_chunked)

[{'parent_id': 'Not correctly extracted',
  'parent_title': 'Not correctly extracted',
  'requirement_tag': 'General',
  'section_id': '0.1',
  'source': 'ISO standards\\ISO_14001_2015(en).pdf',
  'text': '0.1\n'
          'Background Achieving a balance between the environment, society and '
          'the economy is considered essential to meet the needs of the '
          'present without compromising the ability of future generations to '
          'meet their needs. Sustainable development as a goal is achieved by '
          'balancing the three pillars of sustainability. Societal '
          'expectations for sustainable development, transparency and '
          'accountability have evolved with increasingly stringent '
          'legislation, growing pressures on the environment from pollution, '
          'inefficient use of resources, improper waste management, climate '
          'change, degradation of ecosystems and loss of biodiversity. This '
          'has led organizat

50


In [15]:
# Convert sections into Document objects
def To_document_type(documents):
    iso_documents = [
        Document(
            page_content=doc.get("text", ""),  # Default to empty string if 'text' is missing
            metadata={
                "section_id": doc.get("section_id", ""),       
                "parent_id": doc.get("parent_id", ""),         
                "parent_title": doc.get("parent_title", ""),   
                "requirement_tag": doc.get("requirement_tag", ""),
                "source": doc.get("source", "")
            }
        )
        for doc in documents
    ]
    return iso_documents

Iso_chunks_Document_type = To_document_type(all_iso_chunked)

In [ ]:
# check data
for idx, doc in enumerate(Iso_chunks_Document_type):
    print(f"Document {idx+1}:")
    print(f"Page Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-" * 50)

NameError: name 'Iso_chunks_Document_type' is not defined

In [17]:
pprint(Iso_chunks_Document_type)

[Document(metadata={'section_id': '0.1', 'parent_id': 'Not correctly extracted', 'parent_title': 'Not correctly extracted', 'requirement_tag': 'General', 'source': 'ISO standards\\ISO_14001_2015(en).pdf'}, page_content='0.1\nBackground Achieving a balance between the environment, society and the economy is considered essential to meet the needs of the present without compromising the ability of future generations to meet their needs. Sustainable development as a goal is achieved by balancing the three pillars of sustainability. Societal expectations for sustainable development, transparency and accountability have evolved with increasingly stringent legislation, growing pressures on the environment from pollution, inefficient use of resources, improper waste management, climate change, degradation of ecosystems and loss of biodiversity. This has led organizations to adopt a systematic approach to environmental management by implementing environmental management systems with the aim of 

In [20]:
# Save to JSON for further processing
documents_dict = [
    {"metadata": doc.metadata, "page_content": doc.page_content} for doc in Iso_chunks_Document_type
]
    with open("Chunked_files\\ISOsubsectionsAll.json", "w") as outfile:
        json.dump(documents_dict, outfile, indent=4)

    print("saved!")

saved!
